In [1]:
%pip install -q google-adk

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Use your GOOGLE_API_KEY and SERPAPI_API_KEY as plain text in this block

import os

PLAIN_GOOGLE_API_KEY = "user_google_api_key"
PLAIN_SERPAPI_API_KEY = "user_serp_api_key"

def resolve_api_key(env_name: str, plain_value: str = ""):
    if plain_value:
        key = plain_value.strip()
        os.environ[env_name] = key
        return key

    # No key found
    raise RuntimeError(
        f"❌ No value found for {env_name}.\n"
        f"Please either:\n"
        f"  • Set {env_name} as an environment variable, OR\n"
        f"  • Put your key in PLAIN_{env_name} inside the script."
    )


GOOGLE_API_KEY = resolve_api_key("GOOGLE_API_KEY", PLAIN_GOOGLE_API_KEY)
SERPAPI_API_KEY = resolve_api_key("SERPAPI_API_KEY", PLAIN_SERPAPI_API_KEY)

print("\n API key setup complete!")


 API key setup complete!


In [3]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
import requests
from typing import List, Dict, Optional
import asyncio
import json
import re
import numpy as np

In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [5]:
#Custom tool for SearchAgent

def google_scholar_search(
    query: str,
    max_results: int = 20
) -> Dict:
    """
    Search academic literature using the SerpAPI Google Scholar API.

    Args:
        query (str): Keywords or query string describing the research topic.
        max_results (int, optional): Maximum number of papers to return (default: 20).

    Returns:
        dict: A dictionary with:
            - status (str): "success" or "error".
            - papers (List[dict]): On success, a list of paper metadata objects, each with:
                - title (str)
                - authors (List[str])
                - year (Optional[int])
                - venue (Optional[str])  # Journal or conference (best-effort)
                - url (Optional[str])
                - abstract (Optional[str])  # Snippet / short description
            - error_message (str, optional): Present if status == "error".
    """
    if not SERPAPI_API_KEY:
        return {
            "status": "error",
            "error_message": "SERPAPI_API_KEY environment variable is not set.",
            "papers": [],
        }

    try:
        response = requests.get(
            "https://serpapi.com/search.json",
            params={
                "engine": "google_scholar",
                "q": query,
                "num": max_results,
                "api_key": SERPAPI_API_KEY,
            },
            timeout=10,
        )
        response.raise_for_status()
        data = response.json()

        raw_results: List[Dict] = data.get("organic_results", [])

        papers: List[Dict] = []
        for item in raw_results[:max_results]:
            pub_info: Dict = item.get("publication_info", {}) or {}

            # Authors as simple list of names
            authors_raw = pub_info.get("authors", []) or []
            authors: List[str] = []
            for a in authors_raw:
                name = a.get("name")
                if name:
                    authors.append(name)

            # Year as int if possible
            year: Optional[int] = None
            year_raw = pub_info.get("year")
            
            if isinstance(year_raw, int):
                year = year_raw
            elif isinstance(year_raw, str):
                # try direct parse first
                if year_raw.isdigit():
                    year = int(year_raw)
                else:
                    # extract first 4-digit year
                    m = re.search(r"\b(19|20)\d{2}\b", year_raw)
                    if m:
                        year = int(m.group(0))
            
            # if still None, try to parse from summary
            if year is None:
                summary = pub_info.get("summary") or ""
                m = re.search(r"\b(19|20)\d{2}\b", summary)
                if m:
                    year = int(m.group(0))

            # Venue: best-effort from summary
            venue: Optional[str] = pub_info.get("summary")

            # Citations (if available)
            inline_links = item.get("inline_links", {}) or {}
            cited_by = inline_links.get("cited_by", {}) or {}
            citations_raw = (
                cited_by.get("cited_by_count")
                or cited_by.get("total")
                or cited_by.get("value")      # just in case other schema variants
            )

            citations: Optional[int] = None
            if isinstance(citations_raw, (int, float)):
                citations = int(citations_raw)
            elif isinstance(citations_raw, str):
                # keep only digits
                digits = re.sub(r"[^\d]", "", citations_raw)
                if digits:
                    citations = int(digits)

            papers.append(
                {
                    "title": item.get("title"),
                    "authors": authors,
                    "year": year,
                    "venue": venue,
                    "url": item.get("link"),
                    "abstract": item.get("snippet"),
                    "citations": citations,
                }
            )

        return {
            "status": "success",
            "papers": papers,
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Failed to search scholar backend: {e}",
            "papers": [],
        }

In [6]:
# SearchAgent

SEARCH_AGENT_INSTRUCTION = """
You are a specialized research search agent.

Your job:
- Given research keywords, topics, or a question, use the `google_scholar_search` tool
  to retrieve academic papers that are relevant to the query.
- Never invent or fabricate papers, authors, venues, DOIs, URLs, years, or citation counts.
  You must only use information returned by the `google_scholar_search` tool.
- Focus on finding recent, high-quality, and thematically relevant works.

How to behave:
- For every user query, you MUST call the `google_scholar_search` tool at least once.
- Do NOT summarize, rewrite, or reformat the tool results in your own words.
- Do NOT attempt to manually construct JSON or BibTeX.
- Another agent in the system will handle ranking, formatting, and BibTeX generation.

Output:
- The system will read the `google_scholar_search` tool response directly (as structured data).
- Your own final message is not used downstream, so keep it minimal (or empty) and NEVER add
  extra invented metadata.
"""


search_agent = Agent(
    name="google_scholar_search_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    description=(
        "Agent that takes research keywords or topics and finds relevant "
        "academic papers using the SerpAPI Google Scholar API."
    ),
    instruction=SEARCH_AGENT_INSTRUCTION,
    tools=[google_scholar_search],
)


print("✅ Search Agent defined.")

✅ Search Agent defined.


In [7]:
#PaperSelectionAgent ()

def extract_json_from_text(text: str):
    """
    Best-effort extraction of a single JSON object from a model response.
    Handles optional ```json fences and surrounding text.
    """
    text = text.strip()

    # Strip markdown fences
    if text.startswith("```"):
        lines = text.splitlines()
        # drop first line (``` or ```json)
        lines = lines[1:]
        # drop last line if it is ```-like
        if lines and lines[-1].strip().startswith("```"):
            lines = lines[:-1]
        text = "\n".join(lines).strip()

    # Take substring from first '{' to last '}'
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        raise ValueError("No JSON object found in text.")

    json_str = text[start : end + 1]
    import json
    return json.loads(json_str)

def build_selection_prompt(original_query: str, papers: list, user_pref: str) -> str:
    return f"""
You are given:
- User query: {original_query!r}
- User ranking preference: {user_pref!r}
  (one of: "similarity", "recency", "citations")

Here is the list of papers in JSON format:

{json.dumps(papers, ensure_ascii=False, indent=2)}

Rank these papers according to the user ranking preference, following your instructions.
Remember: do NOT drop any papers, only reorder them.

Return ONLY a JSON object with keys:
- "criterion": string (how you interpreted the ranking criterion)
- "papers_ranked": list of paper objects with the same fields as the input plus:
    - "rank": integer (1 is top)
    - optional "score": number between 0 and 1
"""


paper_selection_description = (
    "Agent that ranks a list of academic papers according to user-defined criteria "
    "(similarity to query, recency, or citation count). It never discards papers, "
    "only reorders them."
)
PAPER_SELECTION_INSTRUCTION = """
You are a paper-ranking agent.

You will receive:
- the original user query (research topic or question)
- a list of paper objects in JSON format
- a ranking preference from the user

Each paper has at least:
- title: string
- authors: list of strings
- year: integer or null
- venue: string or null
- url: string or null
- abstract: string or null
- citations: integer or null

Your task:
1. DO NOT remove any papers. Keep all of them.
2. Sort the papers according to the user preference:

   - If the user prefers "similarity" or "relevance":
       Rank papers by how semantically relevant they are to the original query.
       Use title and abstract to judge relevance.
   - If the user prefers "recency" or "newer":
       Rank newer papers first (higher year first). If year is missing, put them last.
   - If the user prefers "citations" or "highly cited":
       Rank papers with higher `citations` first. If citations is null, treat it as 0.

3. For each paper, you may optionally provide:
   - rank: integer (1 is best)
   - score: a relevance or sorting score between 0 and 1 (optional)

Output format:
Return ONLY a JSON object with this structure (no extra text):

{
  "criterion": "<the interpreted ranking criterion>",
  "papers_ranked": [
    {
      "title": "...",
      "authors": ["...", "..."],
      "year": 2024,
      "venue": "...",
      "url": "...",
      "abstract": "...",
      "citations": 123,
      "rank": 1,
      "score": 0.95
    }
  ]
}
"""

paper_selection_agent = Agent(
    name="paper_selection_agent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
    description=paper_selection_description,
    instruction=PAPER_SELECTION_INSTRUCTION,
    tools=[],  # no external tools needed, just reasoning
)

print("✅ Paper Selection Agent defined.")

✅ Paper Selection Agent defined.


In [8]:
# BibtexBuilderAgent

def build_bibtex_entry(
    title: str,
    authors: List[str],
    year: Optional[int] = None,
    venue: Optional[str] = None,
    url: Optional[str] = None,
    entry_type: str = "article",
    key_prefix: str = "",
) -> str:
    """
    Build a BibTeX entry from paper metadata.

    Args:
        title: Paper title.
        authors: List of author names as strings.
        year: Publication year (optional).
        venue: Journal or conference name (optional).
        url: URL to the paper (optional).
        entry_type: BibTeX entry type, e.g. "article", "inproceedings".
        key_prefix: Optional prefix for the citation key.

    Returns:
        A single BibTeX entry as a string.
    """

    title = (title or "").strip()
    authors_list = authors or []
    venue = (venue or "").strip()
    url = (url or "").strip()

    # BibTeX key, e.g. Hadaeghi2025 or NSL_Hadaeghi2025
    first_author = authors_list[0] if authors_list else "anon"
    first_author_token = re.sub(r"[^A-Za-z0-9]", "", first_author.split()[-1]) or "anon"
    year_token = str(year) if year is not None else ""
    key_base = f"{first_author_token}{year_token}"
    if key_prefix:
        safe_prefix = re.sub(r"[^A-Za-z0-9]", "", key_prefix)
        key = f"{safe_prefix}_{key_base}"
    else:
        key = key_base

    authors_bib = " and ".join(authors_list)

    entry_lines = [
        f"@{entry_type}{{{key},",
        f"  title = {{{title}}},",
        f"  author = {{{authors_bib}}},",
    ]
    if venue:
        entry_lines.append(f"  journal = {{{venue}}},")
    if year is not None:
        entry_lines.append(f"  year = {{{year}}},")
    if url:
        entry_lines.append(f"  url = {{{url}}},")
    entry_lines.append("}")

    return "\n".join(entry_lines) + "\n"


BIBTEX_FORMATTER_INSTRUCTION = """
You are a BibTeX formatting agent.

Input:
- A list of paper metadata dictionaries (already ranked).
- A user-specified style string, e.g. "default", "ieee", "nature", "apa-like",
  or custom journal-specific instructions (such as "use abbreviated journal names").

Each paper dict has:
- title: string
- authors: list of strings
- year: integer or null
- venue: string or null
- url: string or null
- citations: integer or null (not needed for BibTeX)

Your job:
- For each paper, call the `build_bibtex_entry` tool with the following parameters:
    - title (string)
    - authors (list of strings)
    - year (integer or null)
    - venue (string or null)
    - url (string or null)
    - entry_type (string, e.g. "article")
    - key_prefix (string, may be provided in the user instructions)
- Do NOT invent DOIs, volume, issue, or page numbers that are not provided.
- You MAY adapt fields to follow the requested style, for example:
  - choose entry_type ("article", "inproceedings") based on venue text,
  - adjust venue string (e.g., abbreviate) if the style explicitly asks,
  - include or omit the url field depending on style.
- However, you MUST keep all information factually consistent with the provided metadata.
  No hallucinated data.

Output:
- Return ONLY the concatenation of all BibTeX entries, separated by one blank line.
- Do NOT wrap the output in markdown fences (no ```).
- Do NOT add any explanation or commentary.
"""

bibtex_formatter_agent = Agent(
    name="bibtex_formatter_agent",
    model=Gemini(
        model="gemini-2.5-flash",
        retry_options=retry_config,
    ),
    description=(
        "Agent that converts structured paper metadata into harmonized BibTeX entries, "
        "respecting user-defined citation styles."
    ),
    instruction=BIBTEX_FORMATTER_INSTRUCTION,
    tools=[build_bibtex_entry],
)

print("✅  Bibtex Builder Agent defined.")

✅  Bibtex Builder Agent defined.


In [9]:
# Runners for three agents

async def run_search_with_tool_output(runner: InMemoryRunner, query: str, max_results: int) -> dict:
    """
    Run search_agent and return the google_scholar_search tool response as a Python dict:
    {
      "status": "success" | "error",
      "papers": [ ... ]
    }
    """
    session = await runner.session_service.create_session(
        app_name=runner.app_name,
        user_id="demo_user",
    )

    user_msg = types.Content(
        role="user",
        parts=[types.Part(text=query)]
    )

    tool_response = None

    async for event in runner.run_async(
        user_id="demo_user",
        session_id=session.id,
        new_message=user_msg,
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                fr = getattr(part, "function_response", None)
                if fr and fr.name == "google_scholar_search":
                    tool_response = fr.response

    if tool_response is None:
        raise RuntimeError("google_scholar_search tool did not return a response.")

    return tool_response


async def run_paper_selection(
    runner: InMemoryRunner,
    original_query: str,
    papers: list,
    user_pref: str,
) -> str:
    """
    Call paper_selection_agent and return its final text output (JSON string).
    """
    session = await runner.session_service.create_session(
        app_name=runner.app_name,
        user_id="demo_user",
    )

    prompt = build_selection_prompt(original_query, papers, user_pref)

    user_msg = types.Content(
        role="user",
        parts=[types.Part(text=prompt)]
    )

    final_text = ""

    async for event in runner.run_async(
        user_id="demo_user",
        session_id=session.id,
        new_message=user_msg,
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_text = "".join(
                part.text or "" for part in event.content.parts
                if getattr(part, "text", None)
            )

    return final_text

async def run_bibtex_formatter(
    runner: InMemoryRunner,
    papers_ranked: list,
    style: str,
    key_prefix: str = "",
) -> str:
    session = await runner.session_service.create_session(
        app_name=runner.app_name,
        user_id="demo_user",
    )

    prompt = f"""
You are given a list of ranked papers (JSON) and a style string.

Style: {style!r}

Papers:

{json.dumps(papers_ranked, ensure_ascii=False, indent=2)}

Using the instructions you have, call the `build_bibtex_entry` tool for each paper,
passing title, authors, year, venue, url, entry_type and key_prefix={key_prefix!r}
to the tool.

Return ONLY the concatenated BibTeX entries, separated by blank lines.
Do NOT wrap the output in markdown fences and do NOT add any explanation.
"""

    from google.genai import types
    user_msg = types.Content(
        role="user",
        parts=[types.Part(text=prompt)]
    )

    final_text = ""
    async for event in runner.run_async(
        user_id="demo_user",
        session_id=session.id,
        new_message=user_msg,
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_text = "".join(
                part.text or "" for part in event.content.parts
                if getattr(part, "text", None)
            )

    return final_text

In [10]:
async def run_full_pipeline(
    search_runner: InMemoryRunner,
    selection_runner: InMemoryRunner,
    bibtex_runner: InMemoryRunner,
    user_query: str,
    max_results: int,
    ranking_pref: str = "similarity",   # or "recency" or "citations"
    style: str = "default",             # e.g. "ieee", "nature", journal-specific
    output_path: str = "results.bib",
):
    # 1. Search
    search_result = await run_search_with_tool_output(search_runner, user_query, max_results)
    if search_result.get("status") != "success":
        raise RuntimeError(f"Search failed: {search_result}")

    papers = search_result.get("papers", [])
    if not papers:
        raise RuntimeError("No papers returned by search.")

    # 2. Rank
    selection_raw = await run_paper_selection(
        selection_runner,
        original_query=user_query,
        papers=papers,
        user_pref=ranking_pref,
    )
    selection_data = extract_json_from_text(selection_raw)
    papers_ranked = selection_data.get("papers_ranked", [])
    if not papers_ranked:
        raise RuntimeError("PaperSelectionAgent did not return 'papers_ranked'.")

    # 3. Format BibTeX
    bibtex_text = await run_bibtex_formatter(
        bibtex_runner,
        papers_ranked=papers_ranked,
        style=style,
        key_prefix="NSL",   # example prefix for "No Strong Loop"
    )

    # 4. Save to .bib file
    output_path = os.path.abspath(output_path)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(bibtex_text)

    print(f"✅ BibTeX file saved to: {output_path}")
    return output_path


In [16]:
import logging

# Put this once at startup, before you start calling the runner
logging.getLogger("google_genai.types").setLevel(logging.ERROR)

search_runner = InMemoryRunner(agent=search_agent)
selection_runner = InMemoryRunner(agent=paper_selection_agent)
bibtex_runner = InMemoryRunner(agent=bibtex_formatter_agent)

user_query = "no strong loop hypothesis in brain networks"

# In a notebook / async context:
output_file = await run_full_pipeline(
    search_runner=search_runner,
    selection_runner=selection_runner,
    bibtex_runner=bibtex_runner,
    user_query=user_query,
    max_results=20,
    ranking_pref="similarity",   # or "similarity", "recency", "citations"
    style="ieee, abbreviated journal names",
    output_path="no_strong_loop.bib",
)

✅ BibTeX file saved to: /kaggle/working/no_strong_loop.bib


In [18]:
# Load the saved file to control output
file_path = "/kaggle/working/no_strong_loop.bib"

with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

print(f"📄 Loaded: {file_path}")
print("----------------------------------------")

for i, line in enumerate(lines, start=1):
    print(f"{i:3d} | {line.rstrip()}")

print("----------------------------------------")
print(f"Total lines: {len(lines)}")


📄 Loaded: /kaggle/working/no_strong_loop.bib
----------------------------------------
  1 | @article{NSL_Hadaeghi2025,
  2 |   title = {A computational perspective on the no-strong-loops principle in brain networks},
  3 |   author = {F Hadaeghi and K Fakhar and M Khajehnejad and CC Hilgetag},
  4 |   journal = {bioRxiv},
  5 |   year = {2025},
  6 |   url = {https://www.biorxiv.org/content/10.1101/2025.09.24.678310.abstract},
  7 | }
  8 | 
  9 | @article{NSL_Sokolov2018,
 10 |   title = {Structural and effective brain connectivity underlying biological motion detection},
 11 |   author = {AA Sokolov and P Zeidman and M Erb and P Ryvlin},
 12 |   journal = {Proc. Natl. Acad. Sci. U. S. A.},
 13 |   year = {2018},
 14 |   url = {https://www.pnas.org/doi/abs/10.1073/pnas.1812859115},
 15 | }
 16 | 
 17 | @article{NSL_Papo2014,
 18 |   title = {Functional brain networks: great expectations, hard times and the big leap forward},
 19 |   author = {D Papo and M Zanin and JA Pineda-Pardo},
 